# Covid testing in US v. Korea

This notebook is used to scrape testing data from US CDC and merge with testing data from Korea CDC.

Sources are found here: 

- [US](https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html) - Scraped using BeautifulSoup
- [Korea](https://www.cdc.go.kr/board/board.es?mid=&bid=0030) - Manually entered from the press releases

In [1]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Scraping US CDC Data

### Step 1: Create a soup object from the home page

In [2]:
# create URL
url = "https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html"

# request url
res = requests.get(url)

### Step 2: Scrape the home page soup for every date

In [3]:
# Create content
soup = BeautifulSoup(res.content, "lxml")

In [4]:
# Find the container
table = soup.find("table", {"class": "table table-header-light nein-scroll"})

# Create the list
dates = []

# Loop through the rows
for row in table.find_all("tr")[1:]: # Skip the header row
    # create the dictionary
    date = {}
    
    cells = row.find_all("td")
    
    # fill the dictionary
    date["date"] = cells[0].text 
    date["CDC"] = cells[1].text
    date["PHL"] = cells[2].text

    # append team to list
    dates.append(date)

# Create a dataframe
df = pd.DataFrame(dates)

## Data cleaning

### Convert date to date time object

In [5]:
df["date"] = df["date"].str.replace("/", "-") # replace slashes with dash
df["date"] = df["date"] + "-2020" # add year to end of date
df["date"] = df['date'].astype('datetime64') # convert to date time object

### Remove last row of data, for missing values

In [6]:
df.drop(index = 59, inplace = True)
df.drop(index = 60, inplace = True)

### Convert cases to numeric

In [7]:
# remove special characters
df["CDC"] = df["CDC"].str.replace("‡", "")
df["PHL"] = df["PHL"].str.replace("‡", "")
df["PHL"] = df["PHL"].str.replace("§", "")

In [8]:
# remove special characters
df["CDC"] = df["CDC"].str.replace(" ", "")
df["PHL"] = df["PHL"].str.replace(" ", "")

In [9]:
df["CDC"] = df['CDC'].astype('int') # convert to date time object
df["PHL"] = df['PHL'].astype('int') # convert to date time object

ValueError: invalid literal for int() with base 10: ''

### Create a total column

In [10]:
# add CDC and Public Health Labs together
df["US"] = df["CDC"] + df["PHL"]

### Read in Korea data

In [12]:
# Read in data. This CSV was manually compiled
df_korea= pd.read_csv("../data/korea_tests.csv")

In [13]:
# Convert to date time object
df_korea["date"] = df_korea['date'].astype('datetime64')

### Find the daily testings numbers

The data on Korea's CDC page gave cumulative totals for testing. I am taking the difference between dates to find the daily testing numbers

In [14]:
# Calculate daily tests
df_korea["KOR"] = df_korea["Korea"].diff(1)

In [15]:
df_korea

,date,Korea,KOR
0,2020-01-18,NaN,NaN
1,2020-01-19,NaN,NaN
2,2020-01-20,NaN,NaN
3,2020-01-21,NaN,NaN
4,2020-01-22,NaN,NaN
5,2020-01-23,NaN,NaN
6,2020-01-24,NaN,NaN
7,2020-01-25,NaN,NaN
8,2020-01-26,NaN,NaN
9,2020-01-27,NaN,NaN


# Merge US and Korea data sets

In [16]:
# merge data
df_total = pd.merge(left = df, right = df_korea, on="date")

In [17]:
# select for overlapping dates
df_total = df_total[(df_total["date"] > "2020-02-08") & (df_total["date"] < "2020-03-14")]

In [18]:
# Only select column totals
df_total = df_total[["date", "US", "KOR"]]

# Export to csv

In [22]:
df_total.to_csv("../data/cdc_tests.tsv", index = False, sep='\t')
df_total.to_csv("../data/cdc_tests.csv", index = False)

# EDA

In [32]:
total_us = df_total["US"].sum()
us_pop = 329414664

In [33]:
total_korea = df_total["KOR"].sum()
korea_pop = 51256929

In [35]:
round(total_us / us_pop, 5)

0.0001

In [34]:
round(total_korea/korea_pop, 6)

0.005065

In [31]:
us_pop > korea_pop

True

In [36]:
round(total_korea/korea_pop, 6) / round(total_us / us_pop, 5)

50.65

In [37]:
round(total_us / us_pop, 5) * 50

0.005